# 함수

In [24]:
import pandas as pd 

# 보여지는 칼럼수 1000개 로 늘이기
pd.set_option('display.max_columns', 1000)

In [25]:
# 1단계: input_df 데이터프레임 만들기
    
def get_input_df():
    import pandas as pd

    # 빈 DataFrame 생성
    columns = ['단지명','전용면적', '가장 가까운 지하철역까지 거리(m)', '환율', '1000m맛집', '1000m문화공간', '1000m문화재', '1000m쇼핑',
               '가장 가까운 버스정류장까지 거리(m)', '일일가격', '지하철역_동대입구역', '지하철역_동묘앞역', '지하철역_명동역',
               '지하철역_버티고개역', '지하철역_상왕십리역', '지하철역_서대문역', '지하철역_서울역', '지하철역_시청역',
               '지하철역_신금호역', '지하철역_신당역', '지하철역_신설동역', '지하철역_약수역', '지하철역_을지로4가역',
               '지하철역_청구역', '지하철역_충무로역', '지하철역_충정로(경기대입구)역', '지하철역_회현(남대문시장)역',
               '지하철역_을지로입구역','지하철역_동대문역사문화공원역']
    idx = [0]
    heng = [['0',0.0, 0.0, 0.0, 0,
              0, 0, 0, 0.0, 0,
              0, 0, 0, 0, 0, 
              0, 0, 0, 0, 0,
              0, 0, 0, 0, 0,
              0, 0,0,0]]
    input_df = pd.DataFrame(data=heng, index = idx, columns=columns )

    # 단지명 , 전용면적, 도로명주소 입력
    apart = input(' 단지명을 입력하세요, ex : 남산센트럴자이 -- ')
    myeonjuk = input(' 전용면적을 입력하세요, ex : 175 -- ')
    # floor = input(' 층수를 입력하세요, ex : 21 -- ')
    juso = input(' 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 -- ')

    # 단지명 : 명동교자
    # 전용면적 : 100
    # 층 : 1
    # 도로명주소 : 중구 명동10길 29

    input_df[['단지명','전용면적','도로명주소']] = apart , myeonjuk, juso
    return input_df

In [26]:
# 2단계: 프로토타입
def output(input_df):
    import pandas as pd
    import numpy as np
    import geopandas as gpd
    from shapely.geometry import Point, Polygon, LineString
    import matplotlib.pyplot as plt
    import matplotlib
    import warnings
    warnings.filterwarnings('ignore')

    # 데이터 가져오기
    metro = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%84%9C%EC%9A%B8_%EC%A7%80%ED%95%98%EC%B2%A0_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
    metro.replace({'서울역역':'서울역'}, inplace=True)

    bus = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EB%B2%84%EC%8A%A4%EC%A0%95%EB%A5%98%EC%9E%A5_%EC%A4%91%EA%B5%AC.csv')
    food = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%A7%9B%EC%A7%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
    munhwa_space = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EA%B3%B5%EA%B0%84_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
    munhwa = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EB%AC%B8%ED%99%94%EC%9E%AC_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
    shopping = pd.read_csv('https://raw.githubusercontent.com/8900j/hhh/main/%EC%A4%91%EA%B5%AC_%EB%AC%B8%ED%99%94%EA%B4%80%EA%B4%91_%EC%87%BC%ED%95%91_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')

    # 좌표 변환
    import requests, json
    def get_location(address):
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        headers = {"Authorization": "KakaoAK 2550728cf88fe76f21185b7b79eddac8"}    
        try:      
            api_json = json.loads(str(requests.get(url,headers=headers).text))
            address = api_json['documents'][0]['address']
            crd = {"lat": str(address['y']), "lng": str(address['x'])}
            address_name = address['address_name']
            return crd
        except:
            crd = {"lat": 0 , "lng" : 0 }
            return crd

    input_df['위경도'] = input_df['도로명주소'].apply(lambda x : get_location(x))
    input_df['위도'] = input_df['위경도'].apply(lambda x : x['lat'])
    input_df['경도'] = input_df['위경도'].apply(lambda x : x['lng'])
    input_df.drop(columns='위경도', inplace=True)

    input_df[['위도','경도']] = input_df[['위도','경도']].astype(float)

    input_df['geometry'] = input_df.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    input_df = gpd.GeoDataFrame(input_df, geometry='geometry')

    input_df.crs = {'init':'epsg:4326'}
    input_df = input_df.to_crs({'init':'epsg:5179'})

    # 지하철
    metro_gdf=gpd.GeoDataFrame(metro,geometry=gpd.points_from_xy(metro.역사위치경도, metro.역사위치위도),crs='epsg:4326')
    def close_metro_dist(point_geometry):
        dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return dist.min()
    def close_metro(point_geometry):
        dist=metro_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return metro.iloc[dist.idxmin()]['역명']
    input_df['가장 가까운 지하철역까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro_dist(x.geometry),axis=1)
    input_df['가장 가까운 지하철역']=input_df.to_crs(epsg=5186).apply(lambda x: close_metro(x.geometry),axis=1)

    # 버스
    bus_gdf=gpd.GeoDataFrame(bus,geometry=gpd.points_from_xy(bus.정류장_경도, bus.정류장_위도),crs='epsg:4326')
    def close_bus_dist(point_geometry):
        dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return dist.min()
    def close_bus(point_geometry):
        dist=bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
        return bus.iloc[dist.idxmin()]['정류장명']
    input_df['가장 가까운 버스정류장까지 거리(m)']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus_dist(x.geometry),axis=1)
    input_df['가장 가까운 버스정류장']=input_df.to_crs(epsg=5186).apply(lambda x: close_bus(x.geometry),axis=1)

    # 버퍼 생성
    # input_df['500버퍼'] = input_df['geometry'].iloc[0].buffer(500)
    # input_df['500버퍼'] = input_df['500버퍼'].astype('geometry')
    input_df['1000버퍼'] = input_df['geometry'].iloc[0].buffer(1000)
    input_df['1000버퍼'] = input_df['1000버퍼'].astype('geometry')

    # 맛집
    food['geometry'] = food.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    food = gpd.GeoDataFrame(food, geometry='geometry')
    food.crs = {'init':'epsg:4326'}
    food = food.to_crs({'init':'epsg:5179'})
    # f = lambda x : np.sum(food.geometry.intersects(x))
    # input_df['500m맛집'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(food.geometry.intersects(x))
    input_df['1000m맛집'] = input_df['1000버퍼'].apply(f)

    # 문화공간
    munhwa_space['geometry'] = munhwa_space.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    munhwa_space = gpd.GeoDataFrame(munhwa_space, geometry='geometry')
    munhwa_space.crs = {'init':'epsg:4326'}
    munhwa_space = munhwa_space.to_crs({'init':'epsg:5179'})
    # f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
    # input_df['500m문화공간'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(munhwa_space.geometry.intersects(x))
    input_df['1000m문화공간'] = input_df['1000버퍼'].apply(f)

    # 문화재
    munhwa['geometry'] = munhwa.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    munhwa = gpd.GeoDataFrame(munhwa, geometry='geometry')
    munhwa.crs = {'init':'epsg:4326'}
    munhwa = munhwa.to_crs({'init':'epsg:5179'})
    # f = lambda x : np.sum(munhwa.geometry.intersects(x))
    # input_df['500m문화재'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(munhwa.geometry.intersects(x))
    input_df['1000m문화재'] = input_df['1000버퍼'].apply(f)

    # 쇼핑
    shopping['geometry'] = shopping.apply(lambda row : Point([row['경도'], row['위도']]), axis= 1)
    shopping = gpd.GeoDataFrame(shopping, geometry='geometry')
    shopping.crs = {'init':'epsg:4326'}
    shopping = shopping.to_crs({'init':'epsg:5179'})
    # f = lambda x : np.sum(shopping.geometry.intersects(x))
    # input_df['500m쇼핑'] = input_df['500버퍼'].apply(f)
    f = lambda x : np.sum(shopping.geometry.intersects(x))
    input_df['1000m쇼핑'] = input_df['1000버퍼'].apply(f)

    # 환율
    import pandas as pd
    url ='https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%ED%99%98%EC%9C%A8%EC%A1%B0%ED%9A%8C'
    tables = pd.read_html(url)[0]
    USD = tables['매매기준율'].iloc[0]
    input_df['환율'] = USD

    # 최종 데이터프레임
    input_final=input_df[['단지명','전용면적','도로명주소','가장 가까운 지하철역까지 거리(m)','가장 가까운 지하철역',
                          '가장 가까운 버스정류장까지 거리(m)','1000m맛집','1000m문화공간',
                          '1000m문화재','1000m쇼핑','환율']]

    return input_final


In [27]:
def input_your_info():    
    input_df=get_input_df()
    return output(input_df)

In [28]:
def make_main_db(): 
    import pandas as pd

    # 빈 DataFrame 생성
    columns = ['단지명','전용면적', '가장 가까운 지하철역까지 거리(m)', '환율',
               '1000m맛집', '1000m문화공간', '1000m문화재', '1000m쇼핑',
               '가장 가까운 버스정류장까지 거리(m)', '일일가격', '지하철역_동대입구역', '지하철역_동묘앞역', '지하철역_명동역',
               '지하철역_버티고개역', '지하철역_상왕십리역', '지하철역_서대문역', '지하철역_서울역', '지하철역_시청역',
               '지하철역_신금호역', '지하철역_신당역', '지하철역_신설동역', '지하철역_약수역', '지하철역_을지로4가역',
               '지하철역_청구역', '지하철역_충무로역', '지하철역_충정로(경기대입구)역', '지하철역_회현(남대문시장)역',
               '지하철역_을지로입구역','지하철역_동대문역사문화공원역']
    idx = [0]
    heng = [['0',0.0, 0.0, 0.0, 0,
              0, 0, 0, 0.0, 0,
              0, 0, 0, 0, 0, 
              0, 0, 0, 0, 0,
              0, 0, 0, 0, 0,
              0, 0,0,0]]
    main_db = pd.DataFrame(data=heng, index = idx, columns=columns )
    main_db.to_csv('main_db.csv', index = False , encoding='utf-8-sig')
    return main_db
main_db = make_main_db()
# main_db = main_db.drop(axis=0)
main_db

,단지명,전용면적,가장 가까운 지하철역까지 거리(m),환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),일일가격,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역,지하철역_을지로입구역,지하철역_동대문역사문화공원역
0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
def reset_db():
    # 빈 DataFrame 생성
    columns = ['단지명','전용면적', '가장 가까운 지하철역까지 거리(m)', '환율',
               '1000m맛집', '1000m문화공간', '1000m문화재', '1000m쇼핑',
               '가장 가까운 버스정류장까지 거리(m)', '일일가격', '지하철역_동대입구역', '지하철역_동묘앞역', '지하철역_명동역',
               '지하철역_버티고개역', '지하철역_상왕십리역', '지하철역_서대문역', '지하철역_서울역', '지하철역_시청역',
               '지하철역_신금호역', '지하철역_신당역', '지하철역_신설동역', '지하철역_약수역', '지하철역_을지로4가역',
               '지하철역_청구역', '지하철역_충무로역', '지하철역_충정로(경기대입구)역', '지하철역_회현(남대문시장)역',
               '지하철역_을지로입구역','지하철역_동대문역사문화공원역']
    idx = [0]
    heng = [['0',0.0, 0.0, 0.0, 0,
              0, 0, 0, 0.0, 0,
              0, 0, 0, 0, 0, 
              0, 0, 0, 0, 0,
              0, 0, 0, 0, 0,
              0, 0,0,0]]
    db = pd.DataFrame(data=heng, index = idx, columns=columns)
    return db

# get_dummies 형식에 맞게 subway 정보 추가하기
def get_dum_subway(db, input_dataframe):
    target = '지하철역_'+ input_dataframe['가장 가까운 지하철역']
    db[target] = 1
    db[['단지명','전용면적','가장 가까운 지하철역까지 거리(m)','환율','1000m맛집','1000m문화공간','1000m문화재','1000m쇼핑','가장 가까운 버스정류장까지 거리(m)']] = input_df[['단지명','전용면적','가장 가까운 지하철역까지 거리(m)','환율','1000m맛집','1000m문화공간','1000m문화재','1000m쇼핑','가장 가까운 버스정류장까지 거리(m)']]
    return db

# main_db 에 추가하기
def add_to_main_db(main_db, tmp_db):
    db = pd.concat([main_db, tmp_db], axis=0)
    db.reset_index(drop =True, inplace= True)
    return db



# 최종 모델

In [35]:
# tmp_db 리셋하기
tmp_db = reset_db()

# input_db 에 소비자 개인 정보 입력하기
input_df = input_your_info()

# 지하철역에 get_dummies 적용된 걸로 tmp_db에 1 채우기
tmp_db= get_dum_subway(tmp_db, input_df)

# main_db 에 tmp_db concat 하기
main_db = add_to_main_db(main_db, tmp_db)

main_db.to_csv('main_db.csv',index=False, encoding="utf-8-sig")

main_db

 단지명을 입력하세요, ex : 남산센트럴자이 --  자이
 전용면적을 입력하세요, ex : 175 --  175
 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 --  중구 퇴계로 235


,단지명,전용면적,가장 가까운 지하철역까지 거리(m),환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),일일가격,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역,지하철역_을지로입구역,지하철역_동대문역사문화공원역
0,0,0.0,0.000000,0.0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,남산 센트럴 자이,175,352.701329,1299.5,63,21,40,9,129.080275,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,남산센트럴자이,175,352.701329,1299.5,63,21,40,9,129.080275,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,신라,150,105.425788,1299.5,50,16,27,10,99.356138,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,자이,175,352.701329,1299.5,63,21,40,9,129.080275,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [34]:
# reset main db
make_main_db()

,단지명,전용면적,가장 가까운 지하철역까지 거리(m),환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),일일가격,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역,지하철역_을지로입구역,지하철역_동대문역사문화공원역
0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 함수 실행

In [108]:
dt = pd.read_csv('dt_train.csv')
dr_cols = ['단지명', '계약년도', '계약일', '경도', '위도', '계약날짜', '500m맛집', '500m문화공간', '500m쇼핑', '500m문화재', '가장 가까운 버스정류장']
dt.drop(dr_cols, axis=1, inplace=True)
dt.tail(1)

,전용면적,지하철거리,환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),일일가격,층_-1,층_1,층_2,층_3,층_4,층_5,층_6,층_7,층_8,층_9,층_10,층_11,층_12,층_13,층_14,층_15,층_16,층_17,층_18,층_19,층_20,층_21,층_22,층_23,층_24,층_25,층_26,층_27,층_28,층_29,층_30,층_31,층_32,층_33,층_34,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역
1093,15.55,76.313787,1234.9,52,17,26,31,102.600962,9.291667,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [64]:
df_origin = pd.read_csv('data_new월세 추가_이상치제거_전용면적_일일가격.csv')
# dr_cols = ['단지명', '보증금','500m음식점수','층', '가장 가까운 버스정류장까지 거리(m)']
# dt.drop(dr_cols, axis=1, inplace=True)
dt= df_origin.copy()
dt.tail(1)

,단지명,전용면적,보증금,월세,층,지하철역,지하철거리,500m음식점수,환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),금리,보증금고려월세
1297,신당KCC스위첸,45.7339,5000.0,150,7,상왕십리역,442.776823,7,1331.5,18,7,4,16,120.540826,3.25,163.541667


In [30]:
dt.corr()

,전용면적,보증금,월세,층,지하철거리,500m음식점수,환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),금리,보증금고려월세
전용면적,1.000000,0.574798,0.520007,0.127265,0.131363,-0.301866,-0.023955,-0.189248,-0.184643,-0.108840,-0.166087,-0.020813,-0.005176,0.709884
보증금,0.574798,1.000000,-0.088264,0.060538,0.121941,-0.111393,-0.036581,-0.074091,-0.053632,0.004993,-0.119603,0.011111,-0.019522,0.298357
월세,0.520007,-0.088264,1.000000,0.133216,0.077007,-0.009380,-0.019969,0.004669,0.005711,0.065953,-0.100198,-0.064383,-0.010588,0.908202
층,0.127265,0.060538,0.133216,1.000000,0.245697,-0.135534,0.034449,-0.226798,-0.258813,-0.246276,0.112217,-0.162109,0.028851,0.160724
지하철거리,0.131363,0.121941,0.077007,0.245697,1.000000,-0.032051,0.010514,-0.120523,-0.167474,-0.204845,0.239263,0.279730,0.024815,0.124382
500m음식점수,-0.301866,-0.111393,-0.009380,-0.135534,-0.032051,1.000000,-0.047954,0.819119,0.785707,0.704625,0.145487,0.044492,-0.047900,-0.054811
환율,-0.023955,-0.036581,-0.019969,0.034449,0.010514,-0.047954,1.000000,-0.058680,-0.059413,-0.054274,-0.041379,0.015521,0.926271,0.075668
1000m맛집,-0.189248,-0.074091,0.004669,-0.226798,-0.120523,0.819119,-0.058680,1.000000,0.975597,0.892591,0.024588,0.109972,-0.061790,-0.028256
1000m문화공간,-0.184643,-0.053632,0.005711,-0.258813,-0.167474,0.785707,-0.059413,0.975597,1.000000,0.928014,-0.069249,0.114316,-0.063979,-0.020860
1000m문화재,-0.108840,0.004993,0.065953,-0.246276,-0.204845,0.704625,-0.054274,0.892591,0.928014,1.000000,-0.371597,0.044632,-0.057097,0.057757


In [65]:
import warnings
warnings.filterwarnings('ignore')

# 1. target = '월세'

In [66]:
# Y = 월세
target = '월세'

# 사용칼럼
dt = dt[['전용면적','지하철역','지하철거리','환율','1000m맛집','1000m문화공간','1000m문화재','1000m쇼핑','가장 가까운 버스정류장까지 거리(m)','금리','월세']]

# x, y split
x = dt.drop(target, axis=1)
y = dt[target]

# get_dummies
dumm_cols = ['지하철역']
x = pd.get_dummies(data =x, columns = dumm_cols, drop_first= True)


In [67]:
x

,전용면적,지하철거리,환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),금리,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역
0,114.9200,358.131262,1185.5,47,17,24,13,392.934357,1.25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,27.2700,329.285132,1185.5,18,7,4,16,155.763152,1.25,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,84.8800,153.773224,1190.8,30,13,28,6,38.785797,1.25,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,84.9791,546.546781,1190.8,47,20,55,4,161.223844,1.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,59.9400,153.773224,1194.3,30,13,28,6,38.785797,1.25,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,63.7400,522.982776,1326.8,22,7,4,23,167.209222,3.25,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1294,30.2477,413.012870,1326.8,18,7,4,22,17.870215,3.25,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1295,18.0824,300.913761,1339.0,143,42,101,11,230.684208,3.25,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1296,57.6300,358.131262,1331.5,47,17,24,13,392.934357,3.25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
# # 정규화
# x = x-x.min() / (x.max()-x.min())

In [68]:
dt.corr()

,전용면적,지하철거리,환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),금리,월세
전용면적,1.000000,0.131363,-0.023955,-0.189248,-0.184643,-0.108840,-0.166087,-0.020813,-0.005176,0.520007
지하철거리,0.131363,1.000000,0.010514,-0.120523,-0.167474,-0.204845,0.239263,0.279730,0.024815,0.077007
환율,-0.023955,0.010514,1.000000,-0.058680,-0.059413,-0.054274,-0.041379,0.015521,0.926271,-0.019969
1000m맛집,-0.189248,-0.120523,-0.058680,1.000000,0.975597,0.892591,0.024588,0.109972,-0.061790,0.004669
1000m문화공간,-0.184643,-0.167474,-0.059413,0.975597,1.000000,0.928014,-0.069249,0.114316,-0.063979,0.005711
1000m문화재,-0.108840,-0.204845,-0.054274,0.892591,0.928014,1.000000,-0.371597,0.044632,-0.057097,0.065953
1000m쇼핑,-0.166087,0.239263,-0.041379,0.024588,-0.069249,-0.371597,1.000000,-0.004194,-0.041625,-0.100198
가장 가까운 버스정류장까지 거리(m),-0.020813,0.279730,0.015521,0.109972,0.114316,0.044632,-0.004194,1.000000,0.012599,-0.064383
금리,-0.005176,0.024815,0.926271,-0.061790,-0.063979,-0.057097,-0.041625,0.012599,1.000000,-0.010588
월세,0.520007,0.077007,-0.019969,0.004669,0.005711,0.065953,-0.100198,-0.064383,-0.010588,1.000000


In [69]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import *

from sklearn.ensemble import RandomForestRegressor

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [94]:
model_rf = RandomForestRegressor(random_state =2022)

cv_score = cross_val_score(model_rf, x_train, y_train, cv=5)

print(cv_score)
print('평균 : ', cv_score.mean())

[0.46564377 0.30735748 0.42099663 0.29774617 0.4737957 ]
평균 :  0.393107952379141


In [ ]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

In [95]:
# 파라미터 선언
params = {'max_depth' : range(5,31)}

model = GridSearchCV(model_rf,
                   params,
                   cv=10,
                   scoring = 'r2')

model.fit(x_train, y_train)

print(model.cv_results_['mean_test_score'])

[0.26881803 0.32193108 0.36480791 0.39772506 0.41476826 0.42541143
 0.43230529 0.4295221  0.43030796 0.42990298 0.42932251 0.42912135
 0.42163972 0.42287254 0.42230419 0.4199755  0.42031963 0.42128126
 0.4187723  0.41982543 0.41959384 0.41660265 0.41678644 0.41647595
 0.41751653 0.41553177 0.41423537 0.41535775 0.41702813 0.41601004
 0.41654967 0.41758801 0.41775948 0.41847981 0.41762706 0.41764088
 0.41736262 0.41711148 0.41714365 0.41724224 0.41721907 0.41721907
 0.41721907 0.41721907 0.41721907 0.41721907 0.41721907 0.41721907
 0.41721907 0.41721907]


# Ridge Regression

In [96]:
# Ridge Regression
import numpy as np
from sklearn.linear_model import Ridge

# training the model
# alpha 값에 변화를 주며 튜닝 0.05, 0.5, 5, 10
tune = [0.001, 0.01,0.05, 0.5, 5, 10]
mae_list = []
for num in tune:
    ridgeReg = Ridge(alpha = num, normalize =True)
    ridgeReg.fit(x_train, y_train)
    pred = ridgeReg.predict(x_test)
    
    # Calculating MAE
    mae = np.mean(abs(pred  - y_test))
    mae_list.append(round(mae,2))
print('alpha = ',tune)
print('mae = ', mae_list)

alpha =  [0.001, 0.01, 0.05, 0.5, 5, 10]
mae =  [47.65, 47.85, 47.85, 49.39, 56.21, 58.0]


# Lasso Regerssion

In [97]:
from sklearn.linear_model import Lasso

# training the model
# alpha 값에 변화를 주며 튜닝 0.05, 0.5, 5, 10
tune = [0.001, 0.01,0.05, 0.5, 5, 10]
mae_list = []
for num in tune:
    lassoReg = Lasso(alpha = num, normalize =True)
    lassoReg.fit(x_train, y_train)
    pred = lassoReg.predict(x_test)
    
    # Calculating MAE
    mae = np.mean(abs(pred  - y_test))
    mae_list.append(round(mae,2))
print(tune)
print(mae_list)

[0.001, 0.01, 0.05, 0.5, 5, 10]
[47.63, 47.98, 48.42, 51.71, 60.45, 60.45]


# Elastic Net Regression

In [98]:
from sklearn.linear_model import ElasticNet

tune = [0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 0.9]
mae_list = []

for num in tune:
    ENreg = ElasticNet(alpha =1, l1_ratio =num, normalize = False)
    ENreg.fit(x_train, y_train)
    pred = ENreg.predict(x_test)
    
    # Calculating MAE
    mae = np.mean(abs(pred  - y_test))
    mae_list.append(round(mae,2))

print(tune)
print(mae_list)

[0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 0.9]
[49.02, 49.02, 49.02, 49.02, 49.04, 49.05, 49.05]


# Deep Learning - Sequential model

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

# Scaling 을 위한 함수
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 딥러닝을 위한 함수들
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session

from tensorflow.keras.optimizers import Adam # 옵티마이저 설정을 위해서

In [100]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [104]:
# 분석단위 구조
nfeatures = x_train.shape[1] # num of columns
nfeatures

60

In [ ]:
# 분석단위 구조
nfeatures = x_train.shape[1] # num of columns

# 메모리 정리 : 처음 부터 다시 모델링 시작할거야~
clear_session()

# 1. Sequential 타입 모델 선언
model = Sequential([Dense(8, input_shape=(nfeatures,), activation='relu'),
                    Dense(4, activation = 'relu'),
                    Dense(1,),
                    ])

# 모델요약
model.summary()

# 2.컴파일
model.compile(optimizer=Adam(learning_rate = 0.001,),
                             loss='mae')

# 3.학습
history = model.fit(x_train, y_train,
                    epochs = 300,                   # 총 data에 대해 100번 학습 반복
                    validation_split=0.2).history

# 4.학습그래프 -> 맨위 함수 참조
dl_history_plot(history)

# 5.검증 및 평가
y_pred = model.predict(x_test)

print(f'MSE  : {mean_squared_error(y_test, y_pred )}')
print(f'RMSE : {mean_squared_error(y_test, y_pred , squared=False)}')
print(f'MAE  : {mean_absolute_error(y_test, y_pred )}')
print(f'MAPE : {mean_absolute_percentage_error(y_test, y_pred )}')
print(f'R2   : {r2_score(y_test, y_pred)}')

In [103]:
dt_train = pd.read_csv('dt_train.csv')
dt_test = pd.read_csv('dt_test.csv')

dt_train

,단지명,전용면적,계약년도,계약일,위도,경도,지하철거리,계약날짜,환율,500m맛집,500m문화공간,500m문화재,500m쇼핑,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m),가장 가까운 버스정류장,일일가격,층_-1,층_1,층_2,층_3,층_4,층_5,층_6,층_7,층_8,층_9,층_10,층_11,층_12,층_13,층_14,층_15,층_16,층_17,층_18,층_19,층_20,층_21,층_22,층_23,층_24,층_25,층_26,층_27,층_28,층_29,층_30,층_31,층_32,층_33,층_34,지하철역_동대입구역,지하철역_동묘앞역,지하철역_명동역,지하철역_버티고개역,지하철역_상왕십리역,지하철역_서대문역,지하철역_서울역,지하철역_시청역,지하철역_신금호역,지하철역_신당역,지하철역_신설동역,지하철역_약수역,지하철역_을지로4가역,지하철역_청구역,지하철역_충무로역,지하철역_충정로(경기대입구)역,지하철역_회현(남대문시장)역
0,현대,79.11,2022-03-01,28,37.560504,127.022971,642.530414,2022/03/28,1218.3,3,0,1,0,27,9,13,16,145.015674,금호베스트빌.래미안하이베르,8.680556,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,래미안하이베르,59.95,2022-09-01,2,37.561038,127.020986,524.953495,2022/09/02,1351.4,7,0,1,0,44,15,23,27,264.007733,동화동,33.541667,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,남산롯데캐슬아이리스,103.49,2022-05-01,27,37.558225,126.981744,310.774962,2022/05/27,1266.4,27,7,12,5,163,48,118,11,85.659904,남산3호터널,75.243056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,쌍용남산플래티넘,166.24,2022-05-01,6,37.559412,126.983220,325.431248,2022/05/06,1261.8,43,5,10,3,175,51,123,11,53.132052,남산3호터널.서울애니메이션센터,53.993056,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,충무,67.77,2022-01-01,27,37.560732,127.000855,458.952760,2022/01/27,1197.0,16,9,15,5,107,37,58,32,159.189499,앰버서더호텔,15.069444,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,청구e편한세상,84.95,2022-04-01,15,37.559960,127.015451,148.070576,2022/04/15,1224.2,4,1,1,0,63,20,29,31,70.234841,청구역,43.750000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1090,남산타운,59.94,2022-09-01,25,37.547167,127.005677,153.773224,2022/09/25,1408.0,10,1,0,1,30,13,28,6,38.785797,버티고개,17.625000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1091,약수하이츠,65.37,2022-09-01,17,37.555192,127.016359,358.131262,2022/09/17,1394.2,7,3,9,0,47,17,24,13,392.934357,신당동대우푸르지오.금호여중,20.729167,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1092,남산타운,84.88,2022-09-01,14,37.547167,127.005677,153.773224,2022/09/14,1374.8,10,1,0,1,30,13,28,6,38.785797,버티고개,32.916667,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
import joblib

In [68]:
air=pd.read_csv('https://raw.githubusercontent.com/8900j/BIG_project/main/Airbnb_Scrap_Data.csv')
air

,이름,주소,최대인원,침실,침대,욕실,가격,detail내용
0,WECOSTAY Myeongdong B,"37.563297, 126.998434",2,1,1,1.0,93750,NaN
1,urbanstay : 명동역 5분거리 / 힙지로 #취사가능,"37.563184, 126.993240",2,1,1,1.0,121260,NaN
2,"[쉼,]고층 편안한 숙소","37.564704, 126.993192",2,1,1,1.0,95437,NaN
3,어반스테이 명동 님이 호스팅하는 레지던스 전체1,"37.564046, 126.992536",2,1,1,1.0,121260,NaN
4,어반스테이 명동 님이 호스팅하는 레지던스 전체2,"37.564326, 126.992977",2,1,1,1.0,121260,NaN
...,...,...,...,...,...,...,...,...
135,#서울역5분 #초고층도시뷰 #욕조 #호텔로비 #장박할인,"37.559342, 126.970736",2,1,1,1.0,79498,NaN
136,"[쉼,]따뜻하고 석양이 아름다운 방, Cosy, Double bed, 힙지로","37.563406, 126.993403",2,1,1,1.0,102000,NaN
137,[Free Meal][Camping][ComoCasa]서울의 중심 풀옵션 레지던스#23,"37.564153, 126.993568",2,1,1,1.0,90000,NaN
138,Room 3. Myeongdong. Namsan. Rooftop,"37.556339, 126.982602",5,3,3,1.0,144219,NaN


In [8]:
#file -> model load
model_rf = joblib.load('saved_model_rf.pkl')
model_rf.fit(x_train, y_train)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RandomForestRegressor(max_features=1.0)

In [9]:
pred = model_rf.predict(x_test)

In [10]:
print('RMSE:', mean_squared_error(y_test, pred, squared=False))
print(' R2:', r2_score(y_test, pred))

RMSE: 8.832161046892814
 R2: 0.74864906179739


In [28]:
input_dt=input_your_info()

del_cols = ['단지명', '500m맛집', '500m문화공간', '500m쇼핑', '500m문화재','도로명주소']
input_dt.drop( columns = del_cols ,axis=1, inplace=True)

input_dt['전용면적'] = input_dt['전용면적'].astype(float)
input_dt['층'] = input_dt['층'].astype(int)

input_dt

 단지명을 입력하세요, ex : 남산센트럴자이 --  남산
 전용면적을 입력하세요, ex : 175 --  175
 층수를 입력하세요, ex : 21 --  21
 도로명 주소를 입력하세요, ex : 중구 퇴계로 235 --  중구 퇴계로 235


,전용면적,층,가장 가까운 지하철역까지 거리(m),가장 가까운 지하철역,가장 가까운 버스정류장까지 거리(m),1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,환율
0,175.0,21,352.701329,충무로역,129.080275,63,21,40,9,1310.0


In [ ]:
# pip install catboost

In [37]:
# train data
data_train = pd.read_csv('data_new월세 추가_이상치제거_전용면적_일일가격.csv')

# 칼럼 drop
del_cols = ['단지명','보증금','500m음식점수','보증금고려월세','금리']
data_train.drop(del_cols, axis =1 ,inplace= True)

data_train.columns = ['전용면적','월세', '층', '지하철역', '가장 가까운 지하철역까지 거리(m)', '환율', '1000m맛집', '1000m문화공간', '1000m문화재','1000m쇼핑', '가장 가까운 버스정류장까지 거리(m)']
data_train['월세'] = data_train['월세']/6
data_train

,전용면적,월세,층,지하철역,가장 가까운 지하철역까지 거리(m),환율,1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,가장 가까운 버스정류장까지 거리(m)
0,114.9200,21.666667,14,신금호역,358.131262,1185.5,47,17,24,13,392.934357
1,27.2700,0.833333,7,상왕십리역,329.285132,1185.5,18,7,4,16,155.763152
2,84.8800,36.666667,15,버티고개역,153.773224,1190.8,30,13,28,6,38.785797
3,84.9791,16.666667,9,충정로(경기대입구)역,546.546781,1190.8,47,20,55,4,161.223844
4,59.9400,19.333333,16,버티고개역,153.773224,1194.3,30,13,28,6,38.785797
...,...,...,...,...,...,...,...,...,...,...,...
1293,63.7400,2.333333,2,신당역,522.982776,1326.8,22,7,4,23,167.209222
1294,30.2477,12.166667,9,상왕십리역,413.012870,1326.8,18,7,4,22,17.870215
1295,18.0824,8.333333,4,시청역,300.913761,1339.0,143,42,101,11,230.684208
1296,57.6300,13.333333,9,신금호역,358.131262,1331.5,47,17,24,13,392.934357


In [40]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   전용면적                  1298 non-null   float64
 1   월세                    1298 non-null   float64
 2   층                     1298 non-null   int64  
 3   지하철역                  1298 non-null   object 
 4   가장 가까운 지하철역까지 거리(m)   1298 non-null   float64
 5   환율                    1298 non-null   float64
 6   1000m맛집               1298 non-null   int64  
 7   1000m문화공간             1298 non-null   int64  
 8   1000m문화재              1298 non-null   int64  
 9   1000m쇼핑               1298 non-null   int64  
 10  가장 가까운 버스정류장까지 거리(m)  1298 non-null   float64
dtypes: float64(5), int64(5), object(1)
memory usage: 111.7+ KB


In [42]:
target = '월세'

x = data_train.drop('월세', axis =1)
y = data_train[target]

dumm_cols = ['지하철역','층']
x = pd.get_dummies(data = x, columns = dumm_cols)
                   
x = x-x.min() / (x.max()-x.min())

In [ ]:
from catboost import CatBoostRegressor, Pool

In [121]:
# # 가변수화
# dumm_cols = ['가장 가까운 지하철역까지 거리(m)','층']

# input_dt = pd.get_dummies(data = input_dt, columns=dumm_cols)
# input_dt

,전용면적,도로명주소,가장 가까운 지하철역,가장 가까운 버스정류장까지 거리(m),1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,환율
0,175.0,중구 퇴계로 235,충무로역,129.080275,63,21,40,9,1310.0


In [113]:
input_dt

,전용면적,도로명주소,가장 가까운 지하철역,가장 가까운 버스정류장까지 거리(m),1000m맛집,1000m문화공간,1000m문화재,1000m쇼핑,환율
0,175.0,중구 퇴계로 235,충무로역,129.080275,63,21,40,9,1310.0


In [112]:
# scaling
input_dt = input_dt - input_dt.min() / (input_dt.max() - input_dt.min())

# pred
predict_price = model_rf.pred(input_df)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
model

In [38]:
your_final_info

In [ ]:
model_rf.predict(

In [ ]:
tmp = input_your_info()
tmp

In [5]:
# 데이터 프레임 초기와
def clear_input_df():
    cnt = 0
    # input_df = pd.DataFrame(columns=range(4))
    # input_df.columns = ['단지명','전용면적','층','도로명주소']
    idx = [cnt]
    heng = [['0',0.0,'0','0']]
    columns = ['단지명','전용면적','층','도로명주소']
    input_df = pd.DataFrame(data=heng, index = idx, columns=columns )
    return input_df

In [11]:
def define_DB():
    DB = clear_input_df()
    return DB

database = define_DB()
database
# def add_to_DB():

,단지명,전용면적,층,도로명주소
0,0,0.0,0,0
